# IMDB VERİLERİNİ KULLANARAK FİLM ÖNERİ ALGORİTMASI

Bu projede imdb film ve özelliklerinin bulunduğu veri setini kullanarak film öneri sistemlerini kuracağız.

Recommendation System (Öneri Sistemi): Bir kullanıcının bir öğeye vereceği 'değerlendirme' ya da 'tercih' miktarının öngörülmesini hedefleyen bir bilgi filtreleme sistemidir.<br>
İçerik Bazlı Sistemler (Content-Based Filtering Systems) : Basitçe, ürünün/ hizmetin içeriğine göre benzer içerikli ürünün önerilmesidir.Örneğin, mavi bir gömlek aldıysanız, gömleğin farklı renginin önerilmesi ya da elektronik müzik seven biriyseniz bir sonraki şarkı önerisinin yine elektronik müzik altyapılı bir şarkı olması örnek olarak verilebilir. <br>
Ben projemde content-based filtering system kullandım.

Kullanacağımız kütüphaneleri ekliyoruz. Verilerimiz üzerinde işlemleri yapmak için "pandas"(veri setimiz üzerinde değişiklikler yapmamızı sağlar) ve "numpy"(matematiksel işlemleri yapabilmemizi sağlar) kütüphanelerini ekliyoruz.

In [58]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [59]:
import pandas as pd 
import numpy as np

## **<font color=green> 1- VERİ YÜKLEME, BİRLEŞTİRME VE TEMİZLEME </font>**
Bu kısımda 2 adet verimiz var onları alıyoruz. "movie_id" ve "id" alanları üzerinden birleştirme yapıyoruz.<br>Ve ihtiyacımız olmayan alanların bir kısmını verimizin dışında bırakıyoruz.

Verisetlerimizi okutuyoruz.

In [60]:
data_credits= pd.read_csv("/content/gdrive/MyDrive/MovieRecommendationSystem/tmdb_5000_credits.csv")
data_movies = pd.read_csv("/content/gdrive/My Drive/MovieRecommendationSystem/tmdb_5000_movies.csv")

Credits dosyamızın ilk 3 satırını görmek için

In [61]:
data_credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


Movies dosyamızın ilk 3 satırını görmek için

In [62]:
data_movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


Shape komutu ile verilerimizin kaç satır kaç sütundan oluştuğunu bakıyoruz.

In [63]:
print("Data Credits:", data_credits.shape)
print("Data Movies:", data_movies.shape)

Data Credits: (4803, 4)
Data Movies: (4803, 20)


Credits verimizin sütunlarına bakıyoruz.

In [64]:
data_credits.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

Movies verimizin sütunlarına bakıyoruz.

In [65]:
data_movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

İki dosyayı birleştirebilmemiz için aynı sütun isimlerine sahip olmaları gerekiyor. 
O yüzden "Movies" verimizin "id" kolonunu "movie_id" olarak değiştiriyoruz.

In [66]:
data_movies.rename(columns = {"id":"movie_id"},inplace=True)

In [67]:
data_movies.columns

Index(['budget', 'genres', 'homepage', 'movie_id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count'],
      dtype='object')

Merge komutu ile data_credits'i data_movies ile movie_id sütunu üzerinden birleştiriyoruz ve data_all'a atıyoruz.

In [68]:
data_all = data_movies.merge(data_credits,on="movie_id")

Shape komutu ile birleştirilmiş veri setimizin satır ve sütunlarının uzunluklarına bakıyoruz.

In [69]:
data_all.shape

(4803, 23)

Veri setimizin ilk 3 satırını görüntülüyoruz.

In [70]:
data_all.head(3)

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


Veri setimizde kullanmak istemediğimiz kolonları belirlemek için sütun isimlerini listeliyoruz.

In [71]:
data_all.columns

Index(['budget', 'genres', 'homepage', 'movie_id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline',
       'title_x', 'vote_average', 'vote_count', 'title_y', 'cast', 'crew'],
      dtype='object')

Drop komutu ile Budget, Homepage ve Status alanlarını veri setimizden çıkarıyoruz.

In [72]:
data_all.drop(columns=["budget","homepage","status"],inplace=True)

Veri setimizin son halini tekrar listeletiyoruz.

In [73]:
data_all.head(3)

,genres,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


## **<font color=green> 2 - İÇERİK TABANLI ÖNERİ SİSTEMİ - CONTENT-BASE RECOMMENDATION SYSTEM </font>**

Film hakkında önizleme yazılarını listeliyoruz.

In [74]:
data_all["overview"].head()

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

İsnull komutu ile önizleme yazıları içerisinde boş değer olup olmadığına bakıyoruz.

In [75]:
data_all.overview.isnull().any()

True

Veri girilmemiş alanlara boş atama yapıyoruz.

In [76]:
data_all['overview'] = data_all['overview'].fillna('')

### -Filmlerimizi tanımlayan kelimeleri ve sıklıklarını hesaplıyıp matrise dönüştürüyoruz. 

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Burada ingilizce "the" , "a" ,"an" gibi kelimeleri temizliyoruz.
tfidf = TfidfVectorizer(stop_words='english')

#Fonksiyonu kullanarak dönüşüm işlemini yapıyoruz
tfidf_matrix = tfidf.fit_transform(data_all['overview'])

#Veri setimizin satır ve sütun sayılarını görüntülüyoruz
tfidf_matrix.shape

(4803, 20978)

In [78]:
data_all['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

### KOSİNÜS BENZERLİĞİNİ KULLANMAK
Benzerlik puanlamasını hesaplamak için birkaç farklı matematiksel yöntem var. Euclidean, Pearson ve Cosine benzerlikleri bunların en bilinenleri.Kosinüs benzerliği doküman uzunluklarına bağlı olmadan benzerliğe bakmamızı sağlıyor. Bu projemizde Kosinüs Benzerliğini kullanarak benzerlik puanlarımızı bulacağız.
<br>Kosinüs Benzerliği Nedir?</br>Kosinüs benzerliği metinler arasındaki benzerliği vektörel olarak ölçmektedir. Metinlerde geçen kelimelerin metinde kaç kez geçtiğini hesaplanır. Daha sonra her metin içerdiği kelimelerle 1 ve 0 şeklinde vektörel olarak ifade edilir. Her metin üç boyutlu uzayda vektörel olarak yerleştirildiğinde aralarındaki kosinüs açısı ne kadar küçük ise metinler birbirlerine o kadar yakındır.

In [79]:
#linear_kernel ekliyoruz
from sklearn.metrics.pairwise import linear_kernel

#cosine benzerliğini hesaplatıyoruz
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [80]:
#indexlerin ve film başlıklarının ters haritasını oluşturuyoruz. Tekrar eden değerleri listeden çıkarıyoruz
indices = pd.Series(data_all.index, index=data_all['original_title']).drop_duplicates()

In [81]:
# Bu fonksiyon ile fonksiyona göndermiş olduğumuz filme en yakın benzerlikte ki 10 filmi geri döndürecek
def get_recommendations(title, cosine_sim=cosine_sim):
    # Girilen filmin indeksini al
    idx = indices[title]

    # Bu filmin diğer tüm filmlerle olan benzerlik puanlarını listele
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Filmleri benzerlik puanlarına göre sırala
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # En çok benzeyen 10 filmin skorunu al
    sim_scores = sim_scores[1:11]

    # Film indekslerini al
    movie_indices = [i[0] for i in sim_scores]

    # En çok benzeyen 10 filmi döndür
    return data_all['original_title'].iloc[movie_indices]


### ÖNERİ FONKSİYONUMUZ TAMAM. ÖRNEK YAPALIM

In [82]:
#The Dark Knight içeriğine benzer 10 film önerisi
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: original_title, dtype: object

In [83]:
#The Godfather içeriğine benzer 10 film önerisi
get_recommendations('Made')

1038          The Infiltrator
1873               Blood Ties
3180       The Way of the Gun
3727              Snabba Cash
4178            Higher Ground
3198             Undiscovered
1809    The Constant Gardener
4188    Dysfunctional Friends
3337            The Godfather
772          The Black Dahlia
Name: original_title, dtype: object